### Import packages

In [35]:
import numpy as np
import cv2
import imutils
from imutils.perspective import four_point_transform
from imutils import contours as cnts

### Answer Key

In [3]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

### Edge Detection

In [168]:
# load the image, convert it to grayscale, blur it
# slightly, then find edges
image = cv2.imread('optical-mark-recognition/images/test_07.png')
orig = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
edged = cv2.Canny(blur, 75, 200)

# display the document outline
cv2.imshow('Original', image)
cv2.imshow('Edged', edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Mark contours (outline) on the OMR sheet

In [169]:
# find contours in the edge map, then initialize
# the contour that corresponds to the document
contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if imutils.is_cv2() else contours[1]
docuContour = None

# ensure that at least one contour was found
if len(contours) > 0:
    # sort the contours according to their size in descending order
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    # loop over the sorted contours
    for c in contours:
        # approximate the contour points
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # if the approximated contour has four points,
        # then we can assume we have found the OMR sheet
        if len(approx) == 4:
            docuContour = approx
            break
            
# Display the found contour
sheetOutline_img = image.copy()
cv2.drawContours(sheetOutline_img, [docuContour], -1, (0, 0, 255), 2)
cv2.imshow('Outline', sheetOutline_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Apply Perspective Transform to get top-view angle

In [170]:
# apply a four point perspective transform to both the original image 
# and grayscale image to obtain a top-down birds eye view of the paper
orig_transform = four_point_transform(image, docuContour.reshape(4,2))
warped = four_point_transform(gray, docuContour.reshape(4,2))

# Display
cv2.imshow('Original Transformed', orig_transform)
cv2.imshow('Grayscale Transformed', warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Thresholding/segmenting the foreground from the background

In [175]:
# apply Otsu's thresholding method to binarize the warped piece of paper
retValue, thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

# Display
cv2.imshow('Segmented', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Find each answer bubble contour (outline) on the OMR sheet

In [172]:
# find contours in the thresholded image, then initialize
# the list of contours that correspond to the bubbles
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if imutils.is_cv2() else contours[1]
bubbleContours = []

# loop over the contours
for c in contours:
    # compute the bounding box of the contour, then use the bounding box to derive the aspect ratio
    (x, y, w, h) = cv2.boundingRect(c)
    aspectRatio = w / float(h)
    
    # in order to label the contour as a bubble, region should be sufficiently wide, 
    # sufficiently tall, and have an aspect ratio approximately equal to 1
    if w >=20 and h >= 20 and aspectRatio >= 0.9 and aspectRatio <= 1.1:
        bubbleContours.append(c)

# Display
bubbleOutline_img = orig_transform.copy()
cv2.drawContours(bubbleOutline_img, bubbleContours, -1, (0, 0, 255), 2)
cv2.imshow('Answer Bubble Outlined', bubbleOutline_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Detect right/wrong marked answer bubbles

In [173]:
# Threshold for checking if an answer is bubbled or not
BUBBLED_THRESHOLD = 650.0

# sort the question contours top-to-bottom, then initialize the total number of correct answers
# Note: "boundingBoxes" is a return value that is not required.
bubbleContours, boundingBoxes = cnts.sort_contours(bubbleContours, method = 'top-to-bottom')
correct = 0

# each question has 5 possible answers, to loop over the
# question in batches of 5
for (ques, ansRowIndex) in enumerate(np.arange(0, len(bubbleContours), 5)):
    # sort the contours for the current question from left to right, 
    # then initialize the index of the bubbled answer
    # Note: "boundingBoxes" is a return value that is not required.
    # By default, it sorts "left-to-right" if method is not specified.
    sorted_contours, boundingBoxes = cnts.sort_contours(bubbleContours[ansRowIndex : ansRowIndex+5])
    bubbled = {}
    
    # loop over the sorted contours
    for (bubbleIndex, c) in enumerate(sorted_contours):
        # construct a mask that reveals only the current "bubble" for the question
        mask = np.zeros(thresh.shape, dtype='uint8')
        cv2.drawContours(mask, [c], -1, 255, -1)
        
        # apply the mask to the thresholded image, 
        # then count the number of non-zero pixels in the bubble area
        mask = cv2.bitwise_and(thresh, thresh, mask = mask)
        total = cv2.countNonZero(mask)
                
        # if the current total has a larger number of total non-zero pixels
        # than the pre-defined threshold, then we are examining the currently bubbled-in answer
        if total > BUBBLED_THRESHOLD:
            bubbled[bubbleIndex] = total
        
    # Get the list of bubbled indices (keys) for a question
    bubbledList = [*bubbled]
    
    # Get the index of the *correct* answer as per the ANSWER_KEY
    k = ANSWER_KEY[ques]
    
    # check to see if any bubble is circled, if not then outline it as Unanswered (Orange)
    if len(bubbledList) == 0:
        cv2.drawContours(orig_transform, [sorted_contours[i] for i in range(0,5)], -1, (0, 165, 255), 3)
        continue
    # check to see if more than one bubbled is filled and outline it as incorrect (Red)
    elif len(bubbledList) > 1:
        cv2.drawContours(orig_transform, [sorted_contours[i] for i in bubbledList], -1, (0, 0, 255), 3)
        continue
    else:
        # check to see if the bubbled answer is correct
        if k == bubbledList[0]:
            correct += 1
        else:
            # draw the outline of the wrongly marked answer (Red) on the OMR sheet
            cv2.drawContours(orig_transform, [sorted_contours[i] for i in bubbledList], -1, (0, 0, 255), 3)
            
    # draw the outline of the correct answer (Green) on the OMR sheet
    cv2.drawContours(orig_transform, [sorted_contours[k]], -1, (0, 255, 0), 3)

### Display the final result

In [176]:
# Calculate the final score based on the correct responses
score = (correct / 5.0) * 100
cv2.putText(orig_transform, 'Final Score = {:.2f}%'.format(score), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
cv2.imshow('Original', image)
cv2.imshow('Result', orig_transform)
cv2.waitKey(0)
cv2.destroyAllWindows()